In [1]:
import os
os.chdir(os.path.join(os.environ["RNB_PLANNING_DIR"], 'src'))

### Make scene

In [2]:
from pkg.controller.combined_robot import *
from pkg.project_config import *

crob = CombinedRobot(robots_on_scene=[
    RobotConfig(0, RobotType.panda, None,
                INDY_IP)]
              , connection_list=[False])
from pkg.geometry.builder.scene_builder import SceneBuilder
s_builder = SceneBuilder(None)
# s_builder.reset_reference_coord(ref_name="floor")
# xyz_rpy_robots = s_builder.detect_items(level_mask=[DetectionLevel.ROBOT])
xyz_rpy_robots = {"panda0": ((0,0,0), (0,0,0))}
crob.update_robot_pos_dict(xyz_rpy_robots=xyz_rpy_robots)
gscene = s_builder.create_gscene(crob)
from pkg.planning.scene import PlanningScene
pscene = PlanningScene(gscene, combined_robot=crob)

connection command:
panda0: False
Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0]


## Preapare background

In [3]:
from pkg.geometry.geometry import *
from pkg.planning.constraint.constraint_actor import Gripper2Tool, PlacePlane

floor = gscene.create_safe(GEOTYPE.BOX, "floor", "base_link", (3,3,0.01), (0,0,0), 
                           rpy=(0,0,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)
# wall = gscene.create_safe(GEOTYPE.BOX, "wall", "base_link", (3,3,0.01), (-0.3,0,0), 
#                            rpy=(0,np.pi/2,0), color=(0.8,0.8,0.8,0.5), display=True, fixed=True, collision=True)

gtems = s_builder.add_robot_geometries(color=(0,1,0,0.5), display=True, collision=True)

gscene.create_safe(gtype=GEOTYPE.SPHERE, name="grip0", link_name="panda0_hand", 
                 dims=(0.01,)*3, center=(0,0,0.112), rpy=(-np.pi/2,0,0), color=(1,0,0,1), display=True, collision=False, fixed=True)
grip0 = pscene.create_binder(bname="grip0", gname="grip0", _type=Gripper2Tool, point=(0,0,0), rpy=(0,0,0))

Please create a subscriber to the marker


## Door knob

In [4]:
from pkg.utils.code_scraps import *

In [5]:
gscene.show_pose(crob.home_pose)

In [6]:
knob = gscene.create_safe(GEOTYPE.BOX, 'knob', link_name="base_link",
                          center=(0.5,0.5,0.5), rpy=(0,-np.pi/2,0), dims=(0.02,0.02,0.02),
                          fixed=True, collision=True, color=(1,0,0,1))

In [7]:
lever_s, knob_s = add_lever(pscene, knob)

### prepare planner

In [8]:
from pkg.planning.motion.moveit.moveit_planner import MoveitPlanner
mplan = MoveitPlanner(pscene)
mplan.update_gscene()

from pkg.planning.task.rrt import TaskRRT
tplan = TaskRRT(pscene)
tplan.prepare()

from pkg.planning.pipeline import PlanningPipeline
ppline = PlanningPipeline(pscene)
ppline.set_motion_planner(mplan)
ppline.set_task_planner(tplan)

from pkg.ui.ui_broker import *
# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(ppline, s_builder)
ui_broker.start_server()

ui_broker.set_tables()

Dash is running on http://0.0.0.0:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production


In [9]:
from pkg.planning.filtering.grasp_filter import GraspChecker
from pkg.planning.filtering.reach_filter import ReachChecker
mplan.motion_filters = [GraspChecker(pscene), ReachChecker(pscene)]
mplan.incremental_constraint_motion = True

   Use a production WSGI server instead.
 * Debug mode: off


### Plan

In [10]:
gscene.show_pose(crob.home_pose)
initial_state = pscene.initialize_state(crob.home_pose)
print(pscene.subject_name_list)
print(initial_state.node)

['knob', 'lever']
('r1', 'knob')


In [11]:
goal_nodes = [('r2','knob')]
ppline.search(initial_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=30, 
              multiprocess=True)

Use 18/36 agents
try: 0 - ('r1', 'knob')->('r2', 'knob')
result: 0 - ('r1', 'knob')->('r2', 'knob') = fail
try: 0 - ('r1', 'knob')->('r1', 'grip0')
try: 0 - ('r1', 'knob')->('r2', 'knob')
error: 2.3
try transition motion
result: 0 - ('r1', 'knob')->('r2', 'knob') = fail
try: 0 - ('r1', 'knob')->('r2', 'knob')
result: 0 - ('r1', 'knob')->('r2', 'knob') = fail
try: 0 - ('r1', 'knob')->('r2', 'knob')
result: 0 - ('r1', 'knob')->('r2', 'knob') = fail
try: 0 - ('r1', 'knob')->('r2', 'knob')
try: 0 - ('r1', 'knob')->('r1', 'grip0')
error: 2.2
try transition motion
result: 0 - ('r1', 'knob')->('r2', 'knob') = fail
try: 0 - ('r1', 'knob')->('r2', 'knob')
result: 0 - ('r1', 'knob')->('r2', 'knob') = fail
try: 0 - ('r1', 'knob')->('r2', 'knob')
result: 0 - ('r1', 'knob')->('r2', 'knob') = fail
try: 0 - ('r1', 'knob')->('r2', 'knob')
try: 0 - ('r1', 'knob')->('r1', 'grip0')
error: 1.8
result: 0 - ('r1', 'knob')->('r2', 'knob') = fail
try: 0 - ('r1', 'knob')->('r2', 'knob')
try: 0 - ('r1', 'knob')

try: 12 - ('r1', 'knob')->('r2', 'knob')
try: 5 - ('r1', 'grip0')->('r2', 'grip0')
try: 1 - ('r1', 'grip0')->('r1', 'knob')
error: 1.1
transition motion tried: True
result: 12 - ('r1', 'knob')->('r2', 'knob') = fail
result: 2 - ('r1', 'knob')->('r2', 'knob') = fail
joint min
result: 1 - ('r1', 'grip0')->('r1', 'knob') = success
constrained motion tried: False
result: 0 - ('r1', 'knob')->('r1', 'grip0') = success
result: 2 - ('r1', 'knob')->('r2', 'knob') = fail
try: 14 - ('r1', 'knob')->('r2', 'knob')
Motion Plan Failure
transition motion tried: True
result: 1 - ('r1', 'grip0')->('r2', 'grip0') = fail
try: 2 - ('r1', 'knob')->('r1', 'grip0')
branching: 0->13 (0.96/30.0 s, steps/err: 57(120.872974396 ms)/0.0014818490952)
result: 0 - ('r1', 'knob')->('r1', 'grip0') = success
try: 5 - ('r1', 'grip0')->('r2', 'grip0')
branching: 1->14 (0.95/30.0 s, steps/err: 1(5.6140422821 ms)/0)
try: 4 - ('r1', 'knob')->('r1', 'grip0')
error: 1.2
branching: 0->15 (0.97/30.0 s, steps/err: 72(268.238067627

transition motion tried: True
try: 13 - ('r1', 'grip0')->('r1', 'knob')
result: 29 - ('r1', 'knob')->('r2', 'knob') = fail
error: 1.1
end
transition motion tried: True
try: 23 - ('r2', 'grip0')->('r1', 'grip0')
try: 1 - ('r1', 'grip0')->('r2', 'grip0')
constrained motion tried: True
result: 6 - ('r1', 'knob')->('r1', 'grip0') = success
result: 2 - ('r1', 'knob')->('r1', 'grip0') = success
branching: 6->31 (1.33/30.0 s, steps/err: 126(260.931015015 ms)/0.00166238674779)
end
error: 1.1
result: 11 - ('r1', 'grip0')->('r2', 'grip0') = success
branching: 11->33 (1.34/30.0 s, steps/err: 80(460.93583107 ms)/1.11106080765e-05)
try: 3 - ('r1', 'grip0')->('r1', 'knob')
branching: 2->32 (1.35/30.0 s, steps/err: 117(233.541965485 ms)/0.00172958495983)
try: 33 - ('r2', 'grip0')->('r2', 'knob')
result: 3 - ('r1', 'grip0')->('r1', 'knob') = success
constrained motion tried: True
result: 5 - ('r1', 'grip0')->('r2', 'grip0') = success
result: 13 - ('r1', 'grip0')->('r1', 'knob') = success
transition mo

constrained motion tried: True
transition motion tried: True
end
try: 13 - ('r1', 'grip0')->('r1', 'knob')
result: 13 - ('r1', 'grip0')->('r2', 'grip0') = success
branching: 13->56 (1.64/30.0 s, steps/err: 80(449.694871902 ms)/9.291202852e-06)
result: 13 - ('r1', 'grip0')->('r1', 'knob') = success
branching: 13->58 (1.65/30.0 s, steps/err: 1(4.60505485535 ms)/0)
result: 35 - ('r1', 'knob')->('r1', 'grip0') = success
constrained motion tried: True
branching: 35->57 (1.65/30.0 s, steps/err: 17(88.7680053711 ms)/0.00110462047788)
result: 7 - ('r1', 'grip0')->('r2', 'grip0') = success
======================= terminated 15: Stop called from other agent  (1.7/30.0) ===============================
end
constrained motion tried: True
======================= terminated 10: Stop called from other agent  (1.7/30.0) ===============================
result: 18 - ('r1', 'grip0')->('r2', 'grip0') = success
======================= terminated 9: Stop called from other agent  (1.7/30.0) ==================

======================= terminated 1: Stop called from other agent  (2.0/30.0) ===============================
======================= terminated 4: required answers acquired  (2.0/30.0) ===============================
========================== FINISHED (2.0 / 30.0 s) ==============================]


In [12]:
snode_schedule = tplan.get_best_schedule()
ppline.play_schedule(snode_schedule)

('r1', 'knob')->('r1', 'grip0')
('r1', 'grip0')->('r2', 'grip0')
('r2', 'grip0')->('r2', 'knob')
('r2', 'knob')->('r2', 'knob')


### push back

In [13]:
term_state = snode_schedule[-1].state
gscene.show_pose(term_state.Q)
print(pscene.subject_name_list)
print(term_state.node)

['knob', 'lever']
('r2', 'knob')


In [14]:
goal_nodes = [("r1",'knob')]
ppline.search(term_state, goal_nodes, max_solution_count=1,
              verbose=True, display=False, dt_vis=0.01, 
              timeout=0.5, timeout_loop=30, 
              multiprocess=False)

try: 0 - ('r2', 'knob')->('r1', 'knob')
result: 0 - ('r2', 'knob')->('r1', 'knob') = fail
try: 0 - ('r2', 'knob')->('r2', 'grip0')
error: 1.3
try transition motion
transition motion tried: True
result: 0 - ('r2', 'knob')->('r2', 'grip0') = success
branching: 0->1 (0.15/30.0 s, steps/err: 52(109.759092331 ms)/0.00155799580102)
try: 0 - ('r2', 'knob')->('r1', 'knob')
result: 0 - ('r2', 'knob')->('r1', 'knob') = fail
try: 1 - ('r2', 'grip0')->('r1', 'grip0')
error: 1.1
collision
constrained motion tried: False
Motion Plan Failure
result: 1 - ('r2', 'grip0')->('r1', 'grip0') = fail
try: 1 - ('r2', 'grip0')->('r2', 'knob')
result: 1 - ('r2', 'grip0')->('r2', 'knob') = success
branching: 1->2 (0.2/30.0 s, steps/err: 1(5.05518913269 ms)/0)
try: 2 - ('r2', 'knob')->('r1', 'knob')
result: 2 - ('r2', 'knob')->('r1', 'knob') = fail
try: 1 - ('r2', 'grip0')->('r2', 'knob')
result: 1 - ('r2', 'grip0')->('r2', 'knob') = success
branching: 1->3 (0.21/30.0 s, steps/err: 1(4.65202331543 ms)/0)
try: 3 -

result: 8 - ('r2', 'knob')->('r2', 'grip0') = success
branching: 8->17 (2.03/30.0 s, steps/err: 12(62.0381832123 ms)/0.00187440862916)
try: 6 - ('r2', 'knob')->('r2', 'grip0')
error: 0.52
try transition motion
transition motion tried: True
result: 6 - ('r2', 'knob')->('r2', 'grip0') = success
branching: 6->18 (2.14/30.0 s, steps/err: 114(102.212905884 ms)/0.00192773749382)
try: 18 - ('r2', 'grip0')->('r2', 'knob')
result: 18 - ('r2', 'grip0')->('r2', 'knob') = success
branching: 18->19 (2.14/30.0 s, steps/err: 1(3.92913818359 ms)/0)
try: 19 - ('r2', 'knob')->('r1', 'knob')
result: 19 - ('r2', 'knob')->('r1', 'knob') = fail
try: 11 - ('r2', 'knob')->('r2', 'grip0')
error: 0.34
try transition motion
transition motion tried: True
result: 11 - ('r2', 'knob')->('r2', 'grip0') = success
branching: 11->20 (2.29/30.0 s, steps/err: 92(121.40583992 ms)/0.00151703524034)
try: 19 - ('r2', 'knob')->('r1', 'knob')
result: 19 - ('r2', 'knob')->('r1', 'knob') = fail
try: 6 - ('r2', 'knob')->('r1', 'kn

In [15]:
snode_schedule = tplan.get_best_schedule(home_pose=crob.home_pose)
ppline.play_schedule(snode_schedule)

('r2', 'knob')->('r2', 'grip0')
('r2', 'grip0')->('r2', 'knob')
('r2', 'knob')->('r2', 'grip0')
('r2', 'grip0')->('r2', 'knob')
('r2', 'knob')->('r2', 'grip0')
('r2', 'grip0')->('r1', 'grip0')
('r1', 'grip0')->('r1', 'knob')
('r1', 'knob')->('r1', 'knob')
